In [ ]:
import pandas as pd
import cv2 


df = pd.read_csv(r"C:\Users\menghang\development\football-server\YOLOX_outputs\yolox_m_mix_det\track_vis\2023_08_07_01_16_32.txt")

In [ ]:
df

In [ ]:
ball = df[df["label"] == "ball"]

In [ ]:
ball

In [ ]:
ball["color"].value_counts()

In [ ]:
ball_possession = []

In [ ]:
import numpy as np

def find_nearest_coordinate(point, coordinates):
    distances = np.linalg.norm(coordinates - point, axis=1)  # Calculate Euclidean distances
    nearest_index = np.argmin(distances)  # Find index of the minimum distance
    nearest_coordinate = coordinates[nearest_index]  # Get the nearest coordinate
    return nearest_coordinate, nearest_index

coordinates = np.array([[2, 4], [1, 3], [5, 6], [0, 1]])
point = np.array([3, 5])

nearest_coordinate , nearest_index = find_nearest_coordinate(point, coordinates)
print("Nearest Coordinate:", nearest_coordinate, nearest_index)

In [ ]:
for i, row in ball.iterrows():
    frame = df[df["frame_id"] == row["frame_id"]]

    print(row)
    ball_center = [row["x_center"], row[" y_center"]]   
    players_centers = []
    for j, f in frame.iterrows(): 
        if(f["label"] != "ball"):
         players_centers.append([f["x_center"], f[" y_center"]])
    
    player, index = find_nearest_coordinate(np.array(ball_center), np.array(players_centers))
    b = frame.iloc[index]
    ball_possession.append(b["color"])
                          
                          
                                        

In [ ]:
occrence = {"white" : 0, "blue":0, "red": 0}
for i in ball_possession:
    occrence[i] += 1


In [ ]:
occrence

In [ ]:
import matplotlib
import PIL

from PIL import Image, ImageFont, ImageDraw
def possession_bar( frame: PIL.Image.Image, origin: tuple, home: float, away:float) -> PIL.Image.Image:
      
        bar_x = origin[0]
        bar_y = origin[1]
        bar_height = 29
        bar_width = 310

        ratio = home
        if ratio < 0.07:
            ratio = 0.07

        if ratio > 0.93:
            ratio = 0.93

        left_rectangle = (
            origin,
            [int(bar_x + ratio * bar_width), int(bar_y + bar_height)],
        )

        right_rectangle = (
            [int(bar_x + ratio * bar_width), bar_y],
            [int(bar_x + bar_width), int(bar_y + bar_height)],
        )

        left_color =  (108, 171, 221)
        right_color = (200, 16, 46)

        frame = draw_counter_rectangle(
            frame=frame,
            ratio=ratio,
            left_rectangle=left_rectangle,
            left_color=left_color,
            right_rectangle=right_rectangle,
            right_color=right_color,
        )

        # Draw home text
        if True:
            home_text = (
                f"{home}"
            )

            frame = text_in_middle_rectangle(
                img=frame,
                origin=left_rectangle[0],
                width=left_rectangle[1][0] - left_rectangle[0][0],
                height=left_rectangle[1][1] - left_rectangle[0][1],
                text=home_text,
                color=(0,0,0)
            )

        # Draw away text
        if True:
            away_text = (
                f"{away}"
            )

            frame = text_in_middle_rectangle(
                img=frame,
                origin=right_rectangle[0],
                width=right_rectangle[1][0] - right_rectangle[0][0],
                height=right_rectangle[1][1] - right_rectangle[0][1],
                text=away_text,
                color=(255,255,255)
            )

        return frame

def text_in_middle_rectangle(
        img: PIL.Image.Image,
        origin: tuple,
        width: int,
        height: int,
        text: str,
        font: PIL.ImageFont = None,
        color=(255, 255, 255),
    ) -> PIL.Image.Image:
        """
        Draw text in middle of rectangle

        Parameters
        ----------
        img : PIL.Image.Image
            Image
        origin : tuple
            Origin of the rectangle (x, y)
        width : int
            Width of the rectangle
        height : int
            Height of the rectangle
        text : str
            Text to draw
        font : PIL.ImageFont, optional
            Font to use, by default None
        color : tuple, optional
            Color of the text, by default (255, 255, 255)

        Returns
        -------
        PIL.Image.Image
            Image with the text drawn
        """

        draw = PIL.ImageDraw.Draw(img)

        if font is None:
            font = PIL.ImageFont.truetype("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/fonts/Gidole-Regular.ttf", size=24)

        w, h = draw.textsize(text, font=font)
        text_origin = (
            origin[0] + width / 2 - w / 2,
            origin[1] + height / 2 - h / 2,
        )

        draw.text(text_origin, text, font=font, fill=color)

        return img
def draw_counter_rectangle(
        frame: PIL.Image.Image,
        ratio: float,
        left_rectangle: tuple,
        left_color: tuple,
        right_rectangle: tuple,
        right_color: tuple,
    ) -> PIL.Image.Image:
        """Draw counter rectangle for both teams

        Parameters
        ----------
        frame : PIL.Image.Image
            Video frame
        ratio : float
            counter proportion
        left_rectangle : tuple
            rectangle for the left team in counter
        left_color : tuple
            color for the left team in counter
        right_rectangle : tuple
            rectangle for the right team in counter
        right_color : tuple
            color for the right team in counter

        Returns
        -------
        PIL.Image.Image
            Drawed video frame
        """

        # Draw first one rectangle or another in orther to make the
        # rectangle bigger for better rounded corners

        if ratio < 0.15:
            left_rectangle[1][0] += 20

            frame = half_rounded_rectangle(
                frame,
                rectangle=left_rectangle,
                color=left_color,
                radius=15,
            )

            frame = half_rounded_rectangle(
                frame,
                rectangle=right_rectangle,
                color=right_color,
                left=True,
                radius=15,
            )
        else:
            right_rectangle[0][0] -= 20

            frame = half_rounded_rectangle(
                frame,
                rectangle=right_rectangle,
                color=right_color,
                left=True,
                radius=15,
            )

            frame = half_rounded_rectangle(
                frame,
                rectangle=left_rectangle,
                color=left_color,
                radius=15,
            )

        return frame
def half_rounded_rectangle(
        img: PIL.Image.Image,
        rectangle: tuple,
        color: tuple,
        radius: int = 15,
        left: bool = False,
    ) -> PIL.Image.Image:
       
        overlay = img.copy()
        draw = PIL.ImageDraw.Draw(overlay, "RGBA")
        draw.rounded_rectangle(rectangle, radius, fill=color)

        height = rectangle[1][1] - rectangle[0][1]
        stop_width = 13

        if left:
            draw.rectangle(
                (
                    rectangle[0][0] + 0,
                    rectangle[1][1] - height,
                    rectangle[0][0] + stop_width,
                    rectangle[1][1],
                ),
                fill=color,
            )
        else:
            draw.rectangle(
                (
                    rectangle[1][0] - stop_width,
                    rectangle[1][1] - height,
                    rectangle[1][0],
                    rectangle[1][1],
                ),
                fill=color,
            )
        return overlay

In [ ]:
from PIL import Image, ImageFont

img = Image.open("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/images/possession_board.png")

possession_bar(img, (30,200), 0.95,0.05)

In [ ]:
cap = cv2.VideoCapture("/Users/menghang/Downloads/YOLOX_outputs/yolox_l_mix_det/track_vis/2023_07_04_17_03_33/a.mov")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # float
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) ) # float
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)


out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

ball_frame = list(ball["frame"])

frame_count = 0

print(len(ball_possession))

ball_found = 0

possession = {"cyan.  " : 0, "r.  ":0 }

possesion_board =  Image.open("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/images/possession_board.png")

while True:     
    success, frame = cap.read()
    if success:
        if frame_count in ball_frame: 
            ball_found += 1

            possession[ball_possession[ball_found-1] ] += 1 
            home = round(possession["cyan.  "] / ball_found, 2)    
            board = np.array( possession_bar(possesion_board, (30,200), home , round(1 - home,2)))
            board = cv2.cvtColor(board,  cv2.COLOR_RGB2BGR)
            print(ball_found)

        

        frame_count += 1
        board = cv2.resize(board, (800,600))
        frame[2200 : 2200 + board.shape[0], 1700 : 1700 + board.shape[1]] = board
        out.write(frame)

    
    else:
        break
out.release()

cap.release()
cv2.destroyAllWindows()
